### Building evasion vs. mean time interval : by Bus - THIS IS A TEST

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
from Utils import TransantiagoConstants
import EvasionBuilder

In [4]:
processed_evasion = EvasionBuilder.runCompleteProcess()

Original number of rows in complete evasion database is: 97373
Number of duplicated rows in complete evasion database is: 6192
Number of collapsed-duplicated rows in complete evasion database is: 3090
Number of rows in complete evasion database without duplicated rows at all is: 91181
Final number of rows in complete evasion database with collapsed duplicated rows is: 94271


C:\Users\leoca_000\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\leoca_000\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


The only non-matched user_code services are: 
D06


* Processing evasion-ddbb before merging

In [5]:
evasion_paradero = processed_evasion[processed_evasion['TP']=='P']
evasion_paradero_first = evasion_paradero[evasion_paradero['N_PUERTA']==1]

* Getting evasion by date-bus

In [6]:
f = {'PATENTE':['count'],'INGRESAN':['sum'], 'NO_VALIDAN':['sum']}
summary = evasion_paradero_first.groupby(['FECHA','PATENTE']).agg(f)
summary.columns=['EVASION_COUNT', 'TOTAL_INGRESAN','TOTAL_NO_VALIDAN']
summary.reset_index(inplace=True)

* Computing evasion rate

In [7]:
summary.loc[:,'EVASION_RATE'] = summary.loc[:,'TOTAL_NO_VALIDAN']/summary.loc[:,'TOTAL_INGRESAN']

* Filtering by common_dates

In [8]:
common_dates = TransantiagoConstants.common_dates

In [9]:
common_dates_timestamp = [pd.to_datetime(x) for x in common_dates]

In [10]:
common_dates_evasion = summary[summary['FECHA'].isin(common_dates_timestamp)]

* Building complete etapas ddbb.

In [11]:
reduced_etapas = pd.DataFrame()

for date in common_dates:
    base_input_path = 'C:/Users/leoca_000/Desktop/Evasion/01_analisis/03_datos/01_SSH/01_TIMESBYBUS/01_ORIGINAL/'
    file_name = date + '-meanByBus.csv'
    grouped_by_bus = pd.read_csv(base_input_path+file_name, encoding = 'latin-1',sep =";")
    reduced_etapas = pd.concat([reduced_etapas,grouped_by_bus])


* Processing etapas ddbb before merging

In [12]:
del reduced_etapas['Unnamed: 0']
reduced_etapas = reduced_etapas.rename(columns = {'fecha':'FECHA', 'sitio_subida':'PATENTE', 'count': 'ETAPAS_COUNT'})
reduced_etapas['FECHA'] = pd.to_datetime(reduced_etapas.FECHA)
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace("-", "")
reduced_etapas['PATENTE'] = reduced_etapas['PATENTE'].str.replace(" ", "")

In [13]:
reduced_etapas.reset_index(drop=True,inplace=True)

In [14]:
type(reduced_etapas.loc[0,'FECHA'])

pandas._libs.tslib.Timestamp

## <strong>TEST MERGING</strong>

The order is:
1. summary > common_dates_evasion.
2. reduced_etapas.
3. evasion_vs_mean = merge (summary, reduced_etapas ) on FECHA and PATENTE > clean_evasion_vs_mean obtained when mean > 0.
4. evasion_vs_mean > within_dates_evasion_vs_mean > common_dates_evasion_vs_mean. 

Finally, common_dates_evasion_vs_mean is used to check is merge is correctly applied

1 - BASICS 

In [15]:
evasion_vs_mean = pd.merge(summary,reduced_etapas, on=['FECHA','PATENTE'], how='left')

In [16]:
evasion_vs_mean_length = len(evasion_vs_mean.index)
print('The total number of rows in evasion vs. mean ddbb is: ' +str(evasion_vs_mean_length))

The total number of rows in evasion vs. mean ddbb is: 1183


* Getting rid of missing values

In [17]:
clean_evasion_vs_mean = evasion_vs_mean[evasion_vs_mean['mean']>0]

In [18]:
clean_evasion_vs_mean.head()

,FECHA,PATENTE,EVASION_COUNT,TOTAL_INGRESAN,TOTAL_NO_VALIDAN,EVASION_RATE,ETAPAS_COUNT,sum,mean,fecha_instalacion
302,2017-03-09,BKWK90,45,94,27,0.287234,353.0,1842.0,5.218130,NaN
303,2017-03-09,CJJH75,48,125,58,0.464000,194.0,1456.0,7.505155,2017-07-03
304,2017-03-09,CJKC18,38,48,7,0.145833,244.0,1052.0,4.311475,2017-07-05
305,2017-03-09,CJRG61,24,33,18,0.545455,328.0,2334.0,7.115854,NaN
306,2017-03-09,FLXB75,20,62,0,0.000000,287.0,2134.0,7.435540,2016-07-26


In [19]:
clean_evasion_vs_mean_length = len(clean_evasion_vs_mean.index)
print('The total number of rows matching in evasion vs. mean ddbb is: ' + str(clean_evasion_vs_mean_length))
print('The percentage of matching is: ' + str(clean_evasion_vs_mean_length/evasion_vs_mean_length*100) + '%')

The total number of rows matching in evasion vs. mean ddbb is: 496
The percentage of matching is: 41.927303465765%


= END BASICS =

2 - NUMBER OF OBSERVATIONS MISMATCHING

In [20]:
total_mismatching = evasion_vs_mean_length - clean_evasion_vs_mean_length
print('The total number of rows mismatching in evasion vs. mean ddbb is: ' + str(total_mismatching))

The total number of rows mismatching in evasion vs. mean ddbb is: 687


= END NUMBER OF OBSERVATIONS MISMATCHING =

3 - CHECKING CONDITIONS

In [21]:
within_dates_evasion_vs_mean = evasion_vs_mean[(evasion_vs_mean['FECHA']>=pd.to_datetime('2017-03-09'))&(evasion_vs_mean['FECHA']<=pd.to_datetime('2017-08-24'))]

In [22]:
within_dates_evasion_vs_mean_length = len(within_dates_evasion_vs_mean.index)
print('The number of rows within common dates range is: ' + str(within_dates_evasion_vs_mean_length))

The number of rows within common dates range is: 777


In [23]:
common_dates_evasion_vs_mean = within_dates_evasion_vs_mean[within_dates_evasion_vs_mean['FECHA'].isin(common_dates_timestamp)]

In [24]:
common_dates_evasion_vs_mean_length = len(common_dates_evasion_vs_mean.index)
print('The total number of rows within common dates range and in common dates is: ' + str(common_dates_evasion_vs_mean_length))

The total number of rows within common dates range and in common dates is: 513


In [25]:
print('IN SUMMARY, THE ORIGINAL NUMBER OF ROWS IS ' + str(evasion_vs_mean_length) + '. FROM THIS, ' + 
     str(within_dates_evasion_vs_mean_length) + ' ARE WITHIN COMMON DATES RANGE. FROM THE LATTER, ' + 
     str(common_dates_evasion_vs_mean_length) + ' ARE WITHIN COMMON DATES RANGE AND IN FACT CORRESPOND TO A COMMON DATE. FINALLY THERE ARE ' +
     str(common_dates_evasion_vs_mean_length-clean_evasion_vs_mean_length) + ' ROWS MISMATCHING THAT SHOULD BE CHECKED IN REDUCED_ETAPAS AND SIMPLIFIED_ETAPAS FILES') 

IN SUMMARY, THE ORIGINAL NUMBER OF ROWS IS 1183. FROM THIS, 777 ARE WITHIN COMMON DATES RANGE. FROM THE LATTER, 513 ARE WITHIN COMMON DATES RANGE AND IN FACT CORRESPOND TO A COMMON DATE. FINALLY THERE ARE 17 ROWS MISMATCHING THAT SHOULD BE CHECKED IN REDUCED_ETAPAS AND SIMPLIFIED_ETAPAS FILES


= END CHECKING CONDITIONS =

4 - CHECKING PROBLEMATIC ROWS USING <strong>common_dates_evasion_vs_mean</strong> AND LOOKING INTO REDUCED_ETAPAS and SIMPLIFIED_ETAPAS FILES

In [26]:
NOT_SEEN_PATENTES_DATES = common_dates_evasion_vs_mean.loc[common_dates_evasion_vs_mean['mean'].isnull(),['FECHA','PATENTE']]
print(len(common_dates_evasion_vs_mean[common_dates_evasion_vs_mean['mean'].isnull()].index))

17


4.1. In reduced etapas > <font color='green'> Passed </font>

In [27]:
i = 0
for index, row in NOT_SEEN_PATENTES_DATES.iterrows():
    #This line makes sense because:
    #1. FECHA: reduced_etapas['FECHA'] of same type of row['FECHA'], see TEST_EvasionPlotter_ByBusByService In [36] and In [37]
    #2. PATENTE: In [12]. Also, PATENTES are always strings, so no problem.
    if reduced_etapas[(reduced_etapas['FECHA']==row['FECHA'])&(reduced_etapas['PATENTE']==row['PATENTE'])].empty:
        i=i+1

print(i)

17


4.2. In simplified_etapas > <font color='green'> Passed </font>

In [28]:
import re
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [29]:
str(row['FECHA'])

'2017-08-22 00:00:00'

In [30]:
j=0
for index,row in NOT_SEEN_PATENTES_DATES.iterrows():
    date = str(row['FECHA']).split(' ')[0]
    patente = re.split('(\d+)',row['PATENTE'])[0] + '-' + re.split('(\d+)',row['PATENTE'])[1]
    etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
    filtered_df = etapas_builder.runLightCompleteProcess()    
    if filtered_df[filtered_df['sitio_subida']==patente].empty: #This lines makes sense because of In [30], line 4.
        j=j+1

print(j)

Not found in turnstile database: 1832615
Not found in turnstile database: 1842842
Not found in turnstile database: 1821215
Not found in turnstile database: 1869904
Not found in turnstile database: 517700
Not found in turnstile database: 517700
Not found in turnstile database: 517700
Not found in turnstile database: 517700
Not found in turnstile database: 517700
Not found in turnstile database: 517700
Not found in turnstile database: 517700
Not found in turnstile database: 517700
Not found in turnstile database: 1764405
Not found in turnstile database: 1637796
Not found in turnstile database: 1867643
Not found in turnstile database: 1873694
Not found in turnstile database: 1877006
17


### CLOSED, <font color='green'> Passed </font>